In [ ]:
# 🚀 Load everything, evaluate the model, and plot confusion matrix + ROC
# -----------------------------------------------

# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Install required packages (only once per session)
!pip install mne tensorflow seaborn --quiet

# Step 3: Import required libraries
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import Sequence
import math
import warnings
warnings.filterwarnings("ignore")

# Step 4: Load the trained model from Drive
model = load_model('/content/drive/MyDrive/EEG_Siena_Seizure_Data/final_seizure_model.h5')
print("✅ Model loaded successfully!")

# 🧠 Redo preprocessing to regenerate X_test and Y_test

# Constants
BASE_DATA_PATH = '/content/drive/MyDrive/EEG_Siena_Seizure_Data/edf_files/'
ANNOTATION_BASE_PATH = '/content/drive/MyDrive/EEG_Siena_Seizure_Data/textfile/'
CHUNK_SIZE_SECONDS = 5
SAMPLING_RATE = 256

import os
import mne

def time_str_to_seconds(t_str):
    h, m, s = map(int, t_str.split('.'))
    return h * 3600 + m * 60 + s

def load_edf(file_path):
    raw = mne.io.read_raw_edf(file_path, preload=True, verbose=False)
    raw.resample(SAMPLING_RATE)
    return raw

def load_seizure_times(annotation_path):
    times = []
    reg_start = None
    with open(annotation_path, 'r') as f:
        for line in f:
            if 'Registration start time:' in line:
                reg_start = time_str_to_seconds(line.strip().split(': ')[1])
            elif 'Seizure start time:' in line and reg_start is not None:
                seizure_time = time_str_to_seconds(line.strip().split(': ')[1])
                times.append(seizure_time - reg_start)
    return times

def segment_eeg(raw, start, end):
    sfreq = int(raw.info['sfreq'])
    data = raw.get_data()
    chunks = []
    for t in range(start, end - CHUNK_SIZE_SECONDS + 1, CHUNK_SIZE_SECONDS):
        s, e = t * sfreq, (t + CHUNK_SIZE_SECONDS) * sfreq
        chunk = data[:, s:e]
        with np.errstate(divide='ignore', invalid='ignore'):
            norm = (chunk - np.mean(chunk, axis=1, keepdims=True)) / np.std(chunk, axis=1, keepdims=True)
            norm = np.nan_to_num(norm)
        chunks.append(norm.T)
    return chunks

def create_dataset_from_file(file_name):
    raw = load_edf(os.path.join(BASE_DATA_PATH, file_name))
    annotation_file = 'Seizures-list-' + file_name.split('-')[0] + '.txt'
    seizure_times = load_seizure_times(os.path.join(ANNOTATION_BASE_PATH, annotation_file))
    total_sec = int(raw.n_times / raw.info['sfreq'])
    preictal, interictal = [], []
    for sz in seizure_times:
        preictal.extend(segment_eeg(raw, max(0, sz - 1800), sz))
        if sz + 1200 < total_sec:
            interictal.extend(segment_eeg(raw, sz + 600, sz + 1200))
    return preictal, interictal

# Load all files
X, Y = [], []
edf_files = [f for f in os.listdir(BASE_DATA_PATH) if f.endswith('.edf')]

for f in edf_files:
    pre, inter = create_dataset_from_file(f)
    X.extend(pre)
    Y.extend([1]*len(pre))
    X.extend(inter)
    Y.extend([0]*len(inter))

X = np.array(X)
Y = np.array(Y)

# Split into test set again (same as training)
from sklearn.model_selection import train_test_split
X_trainval, X_test, Y_trainval, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)

# 💾 Save test data for future use
np.save('/content/drive/MyDrive/EEG_Siena_Seizure_Data/X_test.npy', X_test)
np.save('/content/drive/MyDrive/EEG_Siena_Seizure_Data/Y_test.npy', Y_test)

print("✅ Test data regenerated and saved.")


# Step 5: Load pre-saved test data (update path if needed)
X_test = np.load('/content/drive/MyDrive/EEG_Siena_Seizure_Data/X_test.npy')
Y_test = np.load('/content/drive/MyDrive/EEG_Siena_Seizure_Data/Y_test.npy')

# Step 6: Define test data generator
class EEGDataGenerator(Sequence):
    def __init__(self, x, y, batch_size=8):
        self.x = x
        self.y = y
        self.batch_size = batch_size
        self.indices = np.arange(len(x))

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_idx = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]
        return np.array([self.x[i] for i in batch_idx]), np.array([self.y[i] for i in batch_idx])

test_generator = EEGDataGenerator(X_test, Y_test)

# Step 7: Evaluate the model on test data
results = model.evaluate(test_generator, verbose=0)
print("\n📊 Model Evaluation Metrics:")
print(f"✅ Test Accuracy  : {results[1]:.4f}")
print(f"✅ Test Precision : {results[2]:.4f}")
print(f"✅ Test Recall    : {results[3]:.4f}")

# Step 8: Predictions and true labels
y_prob = model.predict(test_generator).flatten()     # Predicted probabilities
y_pred = (y_prob > 0.5).astype(int)                   # Binary predictions
y_true = np.concatenate([y for _, y in test_generator])  # True labels

# Step 9: Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("🧠 Confusion Matrix")
plt.tight_layout()
plt.show()

# Step 10: ROC Curve & AUC
fpr, tpr, _ = roc_curve(y_true, y_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(6,4))
plt.plot(fpr, tpr, color='darkorange', label=f"ROC Curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], 'k--', label='Random Guess')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("📈 ROC Curve")
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()

print("\n🎉 Visualization complete! Use this for report or presentation confidently.")


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.5 MB/s eta 0:00:00


✅ Model loaded successfully!
✅ Test data regenerated and saved.

📊 Model Evaluation Metrics:
✅ Test Accuracy  : 0.8353
✅ Test Precision : 0.8976
✅ Test Recall    : 0.8718
174/174 ━━━━━━━━━━━━━━━━━━━━ 34s 193ms/step
